In [1]:
%matplotlib inline
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from plots import *
from kls import *
from nats import *
from utils import *
from objectives import *
from torch.distributions.normal import Normal
from torch.distributions.one_hot_categorical import OneHotCategorical as cat
from torch.distributions.relaxed_categorical import RelaxedOneHotCategorical as rcat
from torch.distributions.gamma import Gamma
import sys
import time
import datetime
sys.path.append('/home/hao/Research/probtorch/')
import probtorch
from probtorch.util import expand_inputs
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
N = 30
K = 3
D = 2

## Model Parameters
NUM_SAMPLES = 10
NUM_HIDDEN1 = 16
NUM_HIDDEN2 = 4
NUM_STATS = 3*K
NUM_LATENTS =  K
## Training Parameters
SAMPLE_DIM = 0
BATCH_DIM = 1
BATCH_SIZE = 20
NUM_EPOCHS = 1200
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'gibbs-mu'

In [3]:
Xs = torch.from_numpy(np.load('gmm_fixed_sigma_dataset/obs.npy')).float()
STATES = torch.from_numpy(np.load('gmm_fixed_sigma_dataset/states.npy')).float()
OBS_SIGMA = torch.from_numpy(np.load('gmm_fixed_sigma_dataset/obs_sigma.npy')).float()
Pi = torch.from_numpy(np.load('gmm_fixed_sigma_dataset/init.npy')).float()
NUM_SEQS = Xs.shape[0]
NUM_BATCHES = int((Xs.shape[0] / BATCH_SIZE))

In [4]:
class Enc_mu(nn.Module):
    def __init__(self, num_obs=D,
                       num_hidden1=NUM_HIDDEN1,
                       num_hidden2=NUM_HIDDEN2,
                       num_stats=NUM_STATS,
                       num_latents=NUM_LATENTS):
        super(self.__class__, self).__init__()
        
        self.mus_mu = nn.Sequential(
            nn.Linear(num_stats, num_hidden1),
            nn.Tanh(),
            nn.Linear(num_hidden1, int(0.5*num_hidden1)),
            nn.Tanh(),
            nn.Linear(int(0.5*num_hidden1), num_latents))
        self.mus_log_sigma = nn.Sequential(
            nn.Linear(num_stats, num_hidden1),
            nn.Tanh(),
            nn.Linear(num_hidden1, int(0.5*num_hidden1)),
            nn.Tanh(),
            nn.Linear(int(0.5*num_hidden1), num_latents))
        self.prior_mu = torch.zeros((K, D))
        self.prior_sigma = torch.ones((K, D)) * 3
        if CUDA:
            self.prior_mu = self.prior_mu.cuda()
            self.prior_sigma = self.prior_sigma.cuda()
        
    def forward(self, stat1, stat2, stat3):
        q = probtorch.Trace()
        stats_d1 = torch.cat((stat1, stat2[:, :, :, 0], stat3[:, :, :, 0]), -1) ## S * B * 3K
        stats_d2 = torch.cat((stat1, stat2[:, :, :, 1], stat3[:, :, :, 1]), -1) ## S * B * 3K
        ##
        q_mu1 = self.mus_mu(stats_d1)
        q_sigma1 = self.mus_log_sigma(stats_d1).exp()
        q_mu2 = self.mus_mu(stats_d2)
        q_sigma2 = self.mus_log_sigma(stats_d2).exp()

        q_mu = torch.cat((q_mu1.unsqueeze(-1), q_mu2.unsqueeze(-1)), -1)
        q_sigma = torch.cat((q_sigma1.unsqueeze(-1), q_sigma2.unsqueeze(-1)), -1)

        means = Normal(q_mu, q_sigma).sample()
        q.normal(q_mu, 
                 q_sigma, 
                 value=means, 
                 name='means')

        p = probtorch.Trace()
        p.normal(self.prior_mu, 
                 self.prior_sigma, 
                 value=q['means'], 
                 name='means')    
        return q, p
        
def initialize():
    emc_mu = Enc_mu()
    if CUDA:
        emc_mu.cuda()
    optimizer =  torch.optim.Adam(list(emc_mu.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))    
    return emc_mu, optimizer

In [5]:
def loglikelihood_s(x, z, mu, sigma, D):
    # log-likelihoods
    """
    X should be expanded and repeated along the sample dim
    """
    labels = z.argmax(-1)
    labels_flat = labels.unsqueeze(-1).repeat(1, 1, 1, D)
    x_mu = torch.gather(mu, 2, labels_flat)
#     x_sigma = torch.gather(sigma, 2, labels_flat)
    x_sigma = sigma
    return Normal(x_mu, x_sigma).log_prob(x).sum(-1).sum(-1) # S * B

In [6]:
enc_mu, optimizer = initialize()

In [ ]:
EUBOs = []
ELBOs = []
ESSs = []
KLs_ex = []
KLs_in = []
flog = open('results/log-' + PATH + '.txt', 'w+')
flog.write('EUBO, ELBO, ESS, KLs_ex, KLs_in\n')
flog.close()
obs_sigma = OBS_SIGMA[0].cuda()
time_start = time.time()
for epoch in range(NUM_EPOCHS):
    
    indices = torch.randperm(NUM_SEQS)
    EUBO = 0.0
    ELBO = 0.0
    ESS = 0.0
    KL_ex = 0.0
    KL_in = 0.0
    for step in range(NUM_BATCHES):
        optimizer.zero_grad()
        batch_indices = indices[step*BATCH_SIZE : (step+1)*BATCH_SIZE]
        obs = Xs[batch_indices]
        states = STATES[batch_indices]
        data = shuffler(torch.cat((obs, states), -1)).repeat(NUM_SAMPLES, 1, 1, 1)
        if CUDA:
            obs =data.cuda()[:, :, :, :2]
            states = data.cuda()[:, :, :, 2:]
        stat1, stat2, stat3 = data_to_stats(obs, states, N, K, D)
        q, p = enc_mu(stat1, stat2, stat3)
        log_p_eta = p.log_joint(sample_dims=SAMPLE_DIM, batch_dim=BATCH_DIM)
        log_q_eta = q.log_joint(sample_dims=SAMPLE_DIM, batch_dim=BATCH_DIM)
        obs_mu = q['means'].value.view(NUM_SAMPLES, BATCH_SIZE, K, D)
        ll = loglikelihood_s(obs, states, obs_mu, obs_sigma[0], D)
        log_weights = ll + log_p_eta - log_q_eta
        weights = F.softmax(log_weights, 0).detach()
        eubo = (weights * log_weights).sum(0).mean()
        elbo = log_weights.mean()
        ess = (1. / (weights**2).sum(0)).mean()
        post_mu, post_sigma = Post_mu(stat1, stat2, enc_mu.prior_mu, enc_mu.prior_sigma, obs_sigma, D)
        q_mu = q['means'].dist.loc
        q_sigma = q['means'].dist.scale
        kl_ex, kl_in = kls_normals(q_mu, q_sigma, post_mu, post_sigma)
        eubo.backward()
        optimizer.step()
        EUBO += eubo.item()
        ELBO += elbo.item()
        ESS += ess.item()
        KL_ex += kl_ex.item()
        KL_in += kl_in.item()

    EUBOs.append(EUBO / NUM_BATCHES)
    ELBOs.append(ELBO / NUM_BATCHES)
    ESSs.append(ESS / NUM_BATCHES) 
    KLs_ex.append(KL_ex / NUM_BATCHES)
    KLs_in.append(KL_in / NUM_BATCHES)
    flog = open('results/log-' + PATH + '.txt', 'a+')
    flog.write(str(EUBO/NUM_BATCHES) + ', ' + str(ELBO/NUM_BATCHES) + ', ' + str(ESS/NUM_BATCHES) + ', ' + 
           str(KL_ex/NUM_BATCHES) + ', ' + str(KL_in/NUM_BATCHES) + '\n')

    if epoch % 10 == 0:
        time_end = time.time()
        print('epoch=%d, EUBO=%.3f, ELBO=%.3f, ESS=%.3f, KL_ex=%.3f, KL_in=%.3f (%ds)'
                % (epoch, EUBO/NUM_BATCHES, ELBO/NUM_BATCHES, ESS/NUM_BATCHES, KL_ex/NUM_BATCHES, KL_in/NUM_BATCHES, time_end - time_start))
        time_start = time.time()

epoch=0, EUBO=-277.566, ELBO=-396.903, ESS=1.041, KL_ex=298.248, KL_in=28.001 (2s)
epoch=10, EUBO=-190.968, ELBO=-373.710, ESS=1.050, KL_ex=274.597, KL_in=17.000 (27s)
epoch=20, EUBO=-167.949, ELBO=-338.632, ESS=1.071, KL_ex=239.825, KL_in=20.922 (27s)
epoch=30, EUBO=-157.091, ELBO=-314.367, ESS=1.089, KL_ex=216.658, KL_in=22.595 (27s)
epoch=40, EUBO=-149.103, ELBO=-292.557, ESS=1.107, KL_ex=194.374, KL_in=24.141 (27s)
epoch=50, EUBO=-143.017, ELBO=-275.256, ESS=1.126, KL_ex=176.951, KL_in=20.710 (25s)
epoch=60, EUBO=-138.950, ELBO=-251.761, ESS=1.144, KL_ex=151.898, KL_in=20.015 (27s)
epoch=70, EUBO=-134.295, ELBO=-233.626, ESS=1.165, KL_ex=135.115, KL_in=18.192 (26s)
epoch=80, EUBO=-129.903, ELBO=-218.318, ESS=1.190, KL_ex=119.864, KL_in=16.657 (27s)
epoch=90, EUBO=-126.714, ELBO=-204.586, ESS=1.201, KL_ex=105.870, KL_in=15.273 (21s)
epoch=100, EUBO=-123.188, ELBO=-195.312, ESS=1.210, KL_ex=97.784, KL_in=13.977 (27s)
epoch=110, EUBO=-121.562, ELBO=-191.419, ESS=1.246, KL_ex=92.598, K

In [ ]:
torch.save(enc_mu.state_dict(), 'weights/enc-%s' + PATH)

In [ ]:
def plot_results(EUBOs, ELBOs, ESSs, num_samples, num_epochs, lr):
    fig = plt.figure(figsize=(15, 15))
    fig.tight_layout()
    ax1 = fig.add_subplot(3, 1, 1)
    ax2 = fig.add_subplot(3, 1, 2)
    ax3 = fig.add_subplot(3, 1, 3)
    ax1.plot(EUBOs, 'r', label='EUBOs')
    ax1.plot(ELBOs, 'b', label='ELBOs')

    ax1.tick_params(labelsize=18)
    
    ax3.plot(np.array(ESSs) / num_samples, 'm', label='ESS')
    ax1.set_title('epoch=%d, batch_size=%d, lr=%.1E, samples=%d' % (num_epochs, BATCH_SIZE, lr, num_samples), fontsize=18)
    ax1.set_ylim([-200, -90])
    ax1.legend()
    ax3.legend()
    ax3.tick_params(labelsize=18)
    plt.savefig('train_' + PATH + '.svg')

In [ ]:
plot_results(EUBOs, ELBOs, ESSs, NUM_SAMPLES, NUM_EPOCHS, LEARNING_RATE)

In [ ]:
def sample_single_batch(num_seqs, N, K, D, batch_size):
    indices = torch.randperm(num_seqs)
    batch_indices = indices[0*batch_size : (0+1)*batch_size]
    obs = Xs[batch_indices]
    states = STATES[batch_indices]
    data = shuffler(torch.cat((obs, states), -1)).repeat(NUM_SAMPLES, 1, 1, 1)
    if CUDA:
        obs =data.cuda()[:, :, :, :2]
        states = data.cuda()[:, :, :, 2:]
    return obs, states

obs, states = sample_single_batch(NUM_SEQS, N, K, D, batch_size=25)
stat1, stat2, stat3 = data_to_stats(obs, states, N, K, D)
q, p = enc_mu(stat1, stat2, stat3)

In [ ]:
def plot_samples(obs, states, q, batch_size):
    colors = ['r', 'b', 'gold']
    fig = plt.figure(figsize=(25,25))
    xs = obs[0].cpu()
    zs = states[0].cpu()
    means_mean = q['means'].dist.loc[0].cpu().data.numpy()

    for b in range(batch_size):
        ax = fig.add_subplot(int(batch_size / 5), 5, b+1)
        x = xs[b].data.numpy()
        z = zs[b].data.numpy()
#         covs = np.zeros((K, D, D))
        mu = means_mean[b].reshape(K, D)
        assignments = z.argmax(-1)
        for k in range(K):
            cov_k = np.diag(np.ones(2))
            xk = x[np.where(assignments == k)]
            ax.scatter(xk[:, 0], xk[:, 1], c=colors[k])
#             ax.scatter(x[:, 0], x[:, 1])
            plot_cov_ellipse(cov=cov_k, pos=mu[k], nstd=2, ax=ax, alpha=0.2, color=colors[k])
        ax.set_ylim([-10, 10])
        ax.set_xlim([-10, 10])
    plt.savefig('results/modes' + PATH + '.svg')

In [ ]:
plot_samples(obs, states, q, batch_size=25)

In [ ]:
states[0,0].argmax(-1)